# Import Libs

In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

# Load openAI key

In [2]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

# Initializing the model

In [3]:
openai = OpenAI()
OPENAI_MODEL = "gpt-4o"

# System Prompt

In [34]:
system_message = "You are an assistant that reimplements a programming code from a language to another in high performance. "
system_message += "Respond only with a programming code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The code response needs to produce an identical output in the fastest possible time. "
system_message += "The code should be able to run with no need to any modification by the user which means you have to include all the necessary packages and fix any error that can be raised. "
system_message += "NOTE!! DO NOT PUT THE CODE BETWEEN ```"

# User Prompt

In [6]:
def get_user_prompt(source_lang, target_lang, code):
    user_message = f"Rewrite this {source_lang} code in {target_lang} with the fastest possible implementation that produces identical output in the least time if possible. "
    user_message += f"Respond only with {target_lang} code; do not explain your work other than a few comments. "
    user_message += f"Code should be clean, clear and complete with all imports needed. the {source_lang} code is:\n\n"
    user_message += code
    
    return user_message

# Prepare the Prompt

In [7]:
def get_prompt(source_lang, target_lang, code):
    system = system_message
    user = get_user_prompt(source_lang, target_lang, code)

    return [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user}
    ]

# Get Model Response

In [30]:
def convert_gpt(source_lang, target_lang, code):
    prompt = get_prompt(source_lang, target_lang, code)
    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=prompt, stream=True)
    reply = ''
    
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ''
        
        yield reply

# Interface

In [66]:
custom_css = """
/* Button styling */
.white-bg {
   background: linear-gradient(135deg, #667eea, #764ba2) !important;
   color: white !important;
   border: none !important;
   border-radius: 12px !important;
   font-weight: 600 !important;
   padding: 15px 30px !important;
   box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3) !important;
}

.white-bg:hover {
   transform: translateY(-2px) !important;
   box-shadow: 0 8px 25px rgba(102, 126, 234, 0.5) !important;
}

/* Row styling */
.row {
   background: rgba(255, 255, 255, 0.15) !important;
   border-radius: 15px !important;
   padding: 20px !important;
   margin: 10px 0 !important;
   backdrop-filter: blur(10px) !important;
   border: 1px solid rgba(255, 255, 255, 0.2) !important;
}
"""

In [67]:
with gr.Blocks(css=custom_css) as UI:
    with gr.Row():
        source_lang = gr.Dropdown(label='Select programming language (From)', choices=["Python", "JavaScript", "C++", "C#", "Java", "Rust"], value='Python')
        target_lang = gr.Dropdown(label='Select programming language (To)', choices=["Python", "JavaScript", "C++", "C#", "Java", "Rust"], value='C++')
    with gr.Row():
        source_code = gr.Textbox(label="Source Code", lines=10)
        target_code = gr.Textbox(label="Target Code", lines=10)
    with gr.Row():
        convert = gr.Button('Convert Code', elem_classes=['white-bg'])

    convert.click(convert_gpt, inputs=[source_lang, target_lang, source_code], outputs=[target_code])

UI.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7884
* To create a public link, set `share=True` in `launch()`.
